In [1]:
import json
import os
from dataclasses import dataclass, field

import spotify
from dotenv import load_dotenv

In [2]:
load_dotenv(".env.local")
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

In [3]:
client = spotify.Client(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)

In [4]:
async def get_all_albums(artist_id: str, *, include_groups: str | None = None, market="US"):
    offset = 0
    total = (await client.http.artist_albums(
        artist_id, limit=1, offset=0, include_groups=include_groups, market=market
    ))["total"]
    count = 0
    while count < total:
        data = await client.http.artist_albums(
            artist_id, limit=50, offset=offset, include_groups=include_groups, market=market
        )
        offset += 50
        if not data["items"]:
            break
        for album in (spotify.Album(client, item) for item in data["items"]):
            yield album
            count += 1

In [5]:
@dataclass(unsafe_hash=True)
class Album:
    name: str
    release_date: str
    group: str
    type: str
    id: str
    tracks: list["Track"] = field(default_factory=list, init=False, hash=False, repr=False)

@dataclass(unsafe_hash=True)
class Track:
    name: str
    album: Album = field(hash=False)
    id: str
    uri: str
    replaces: set["Track"] = field(default_factory=set, init=False, hash=False)
    replacements: set["Track"] = field(default_factory=set, init=False, hash=False)
    sorted_replacements: list["Track"] = field(default_factory=list, init=False, hash=False)


async def load_tracks() -> tuple[list[Album], list[Track]]:
    album_list = []
    track_list = []

    TAYLOR_SWIFT_ID = "06HL4z0CvFAxyc27GXpf02"
    async for album in get_all_albums(TAYLOR_SWIFT_ID, include_groups="album,single,compilation", market="US"):
        print(album.name)
        album_list.append(Album(album.name, album.release_date, album.group, album.type, album.id))

    album_list.sort(key=lambda a: a.release_date, reverse=True)

    for album in album_list:
        print(album.name)
        album.tracks = [
            Track(track["name"], album, track["id"], track["uri"])
            for track in (await client.http.album_tracks(album.id, limit=50))["items"]
        ]
        track_list.extend(album.tracks)

    with open("taylor_tracks.json", "w") as f:
        json.dump([
            {"name": album.name, "release_date": album.release_date, "id": album.id, "tracks": [{"name": track.name, "id": track.id} for track in album.tracks]}
            for album in album_list
        ], f, indent=2)

    return album_list, track_list


albums, tracks = await load_tracks()

Speak Now (Taylor's Version)
Midnights (The Til Dawn Edition)
Midnights (3am Edition)
Midnights
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
evermore
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore (deluxe version)
folklore
Lover
reputation
reputation Stadium Tour Surprise Song Playlist
1989 (Deluxe Edition)
1989
Red (Deluxe Edition)
Red
Speak Now World Tour Live
Speak Now (Deluxe Edition)
Speak Now
Fearless Platinum Edition
Fearless
Live From Clear Channel Stripped 2008
Taylor Swift
Lavender Haze (Acoustic Version)
Lavender Haze (Remixes)
Lavender Haze (Felix Jaehn Remix)
Anti-Hero (Acoustic Version)
Anti-Hero (ILLENIUM Remix)
Anti-Hero (Remixes)
Anti-Hero (feat. Bleachers)
Carolina (From The Motion Picture “Where The Crawdads Sing”)
All Too Well (10 Minute Version) (The Short Film)
This Love (Taylor’s Version)
The Joker And The Queen (feat. Taylor Swift)
Message In A Bottle (Fat Max G Remix) (Taylor’s Versio

In [6]:
def normalize_name(name: str):
    return name \
        .replace("\u2019", "'") \
        .replace("SuperStar", "Superstar") \
        .replace("I Knew You Were Trouble.", "I Knew You Were Trouble")

def get_song_name(name: str):
    return name.split("(", 1)[0].strip()

for track in tracks:
    track.replaces = set()
    track.replacements = set()

for track in tracks:
    name = normalize_name(track.name)
    if "(Taylor's Version)" in name and "(From The Vault)" not in name:
        song_name = get_song_name(name)

        for stolen in tracks:
            stolen_name = normalize_name(stolen.name)
            if stolen_name.startswith(song_name) and " Version)" not in stolen_name:  # Version) also catches ATW versions
                track.replaces.add(stolen)
                stolen.replacements.add(track)
        if not track.replaces:
            print("No stolen found:", track.name, "->", song_name)


for stolen in tracks:
    song_name = normalize_name(stolen.name)
    stolen.sorted_replacements = sorted(
        stolen.replacements,
        key=lambda track: (
            song_name == get_song_name(normalize_name(track.name)),
            song_name in track.album.name,
            track.album.type == "album",
            track.name == track.album.name
        ),
        reverse=True
    )

for album in albums:
    print("\n\n\n###", album.name, end="\n\n")
    for track in album.tracks:
        print(track.name)
        for replacement in track.sorted_replacements:
            print("  ->", replacement.name, "–", replacement.album)
        if (
            not track.sorted_replacements and
            "(Taylor's Version)" not in album.name and
            album.release_date < "2019-08-20"
        ):
            print("  NO REPLACEMENTS")


replacements = {}
for stolen in tracks:
    if stolen.sorted_replacements:
        replacements[stolen.id] = {
            "__name ": stolen.name,
            "__album": stolen.album.name,
            "__album_type": stolen.album.type,
        }
        if "live" in stolen.name.lower():
            replacements[stolen.id]["is_live"] = True
        if "remix" in stolen.name.lower():
            replacements[stolen.id]["is_remix"] = True
        replacements[stolen.id]["replacements"] = [
            {
                "__name ": track.name,
                "__album": track.album.name,
                "__album_type": track.album.type,
                "id": track.id,
            }
            for track in stolen.sorted_replacements
        ]
with open("taylorsversion.json", "w") as f:
    json.dump(replacements, f, indent=2)


No stolen found: Message In A Bottle (Fat Max G Remix) (Taylor’s Version) -> Message In A Bottle
No stolen found: Eyes Open (Taylor's Version) -> Eyes Open
No stolen found: Safe & Sound (feat. Joy Williams and John Paul White) (Taylor’s Version) -> Safe & Sound
No stolen found: Safe & Sound (feat. Joy Williams and John Paul White) (Taylor’s Version) -> Safe & Sound
No stolen found: Eyes Open (Taylor's Version) -> Eyes Open



### Speak Now (Taylor's Version)

Mine (Taylor's Version)
Sparks Fly (Taylor’s Version)
Back To December (Taylor's Version)
Speak Now (Taylor's Version)
Dear John (Taylor's Version)
Mean (Taylor's Version)
The Story Of Us (Taylor's Version)
Never Grow Up (Taylor's Version)
Enchanted (Taylor's Version)
Better Than Revenge (Taylor's Version)
Innocent (Taylor's Version)
Haunted (Taylor's Version)
Last Kiss (Taylor's Version)
Long Live (Taylor's Version)
Ours (Taylor’s Version)
Superman (Taylor’s Version)
Electric Touch (feat. Fall Out Boy) (Taylor’s Version) (From Th